## 데이터 이해 및 탐색

### Feature
  - Class 1이 사기이고, 0은 정상 데이터를 의미  
  - Amount 는 신용카드 사용 금액, Time은 의미 없는 피처로 제거  
  - 결측치는 없는 깔끔한 데이터
  
  
### 데이터의 분포와 특성을 파악
  - 파이 그래프를 이용하여 Class 컬럼의 데이터 분포 특성을 확인  
  - 정상인 데이터가 사기 데이터에 비해 많은 불균형 데이터임을 확인  

  
## 데이터 전처리 과정

### 데이터 정제  
  - 결측치는 없어서 제거하지 않음  
  - 이상치 처리로는 상관계수를 확인하여 상관 정도가 낮은 피처 V14와 V17을 발견하였으나 V14의 이상치만 제거하기로 결정  
  (V14만 이상치 제거 후 선형 회귀 roc-auc: 0.9376)   
  (V14, V17 이상치 제거 후 선형 회귀 roc-auc: 0.9376)  
  (V17만 이상치 제거 후 선형 회귀 roc-auc: 0.9376)  
  
### 데이터 스케일링 및 인코딩 
   - 스케일을 줄이고 분포를 정규화하기 위해 Amount 컬럼에 대해서 Amount_Scaled 새로운 컬럼을 만들었음
  
### 피처 엔지니어링(파생 변수)
   - V1~V28은 PCA를 통해 생성된 주성분들이므로 각 V 피처 간의 상관성이 이미 최소화되어 있음 -> corr 그래프를 이용하여 확인
   - 따라서 파생 변수는 따로 만들지 않기로 결정

## 모델 선택 및 비교
**로지스틱 회귀와 XGBoost에서 하이퍼파라미터를 적용하지 않은 모델이 성능이 좋게 나와서 파라미터 적용하지 않은 전제임.**

    - 로지스틱 회귀
    선택 이유: 기초 모델
    결과:  
    smote 적용 전: 0.9376  
    smote 적용 후: 0.9725
    
    - LightGBM
    선택 이유: 소수(적은 수의 사기 데이터)도 학습을 강화할 수 있음
    결과:
    smote 적용 전: 0.9769  
    smote 적용 후: 0.9793 
    
    - XGBoost
    선택 이유: 불균형 분류 문제에서 성능을 잘 뽑기 때문, 하이퍼파라미터로 과적합 방지
    결과:
    smote 적용 전: roc-auc: 0.9800  
    smote 적용 후: roc-auc: 0.9883
    smote 적용 후 + 하이퍼파라미터: 0.9779  
    
    - Ensemble
    선택 이유: 다양한 모델을 조합한 앙상블은 하나의 모델보다 새로운 데이터에 더 안정적으로 대응할 가능성이 높다고 판단.
    결과:
    smote 적용 전: 0.9596
    smote 적용 후: 0.9618

## import 및 버전 충돌 문제 해결

In [ ]:
!pip install -U scikit-learn==1.4.2 imbalanced-learn==0.12.0

In [ ]:
pip uninstall -y autogluon-tabular autogluon-multimodal autogluon-core autogluon-features nilearn bigframes mlxtend

In [ ]:
pip install scikit-learn==1.2.2 imbalanced-learn==0.10.1

In [ ]:
# pip install scikit-learn==1.2.2 imbalanced-learn==0.10.1

In [ ]:
# pip install scikit-learn==1.2.2 imbalanced-learn==0.10.1

In [ ]:
# pip install scikit-learn==1.3.2 imbalanced-learn==0.11.0

In [ ]:
pip show scikit-learn

In [ ]:
pip show imbalanced-learn

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score
from lightgbm import LGBMClassifier
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

## 데이터 불러오기

In [ ]:
train = pd.read_csv("/kaggle/input/modu-ds-4-credit-card-fraud-detection/train.csv")
test = pd.read_csv("/kaggle/input/modu-ds-4-credit-card-fraud-detection/test.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# 170882 를 기점으로 그 뒤 시점으로 자름

train["id"].max(), test["id"].min()

In [ ]:
train.head()

In [ ]:
train.info()

## Class컬럼의 0과 1 분포도 확인

In [ ]:
train["Class"].value_counts(normalize=True)*100

In [ ]:
import matplotlib.pyplot as plt

class_counts = train["Class"].value_counts(normalize=True) * 100
labels = class_counts.index.map({0: "normal transaction(0)", 1: "fraudulent transaction (1)"})

plt.figure(figsize=(5, 5))
plt.pie(class_counts, labels=labels, autopct="%.3f%%")
plt.axis('equal')  # 동그란 원 유지
plt.show()


In [ ]:
pd.set_option('display.max_columns', None)
train.describe()

### 사기 건수가 정상 건수에 비해 데이터가 극히 적기 때문에 오버 샘플링 방법을 통해 충분한 데이터를 확보한다.
### 동일한 데이터를 단순히 증식하는 방법은 과적합이 되기 때문에 대표적으로 SMOTE 방법을 이용한다.

## 함수 모음

### 전처리 과정 함수

In [ ]:
def get_preprocessed_df(df=None, train=True):
    df_copy = df.copy()
    
    # 스케일을 줄이고 분포를 정규화 -> 변환된 결과를 "Amount_Scaled"이라는 새로운 컬럼으로 맨 앞(0번째 열)에 삽입
    amount_n = np.log1p(df_copy["Amount"])
    df_copy.insert(0, "Amount_Scaled", amount_n)
    
    #불필요한 컬럼 제거
    df_copy.drop(["Time", "Amount"], axis=1, inplace=True)

    # 이상치 제거
    outlier_index = get_outlier(df=df_copy, column="V14", weight=1.5)
    df_copy.drop(outlier_index, axis=0, inplace=True)
    
    return df_copy

In [ ]:
 
def get_preprocessed_df2(df=None, train=True):
    df_copy = df.copy()
    
    # 스케일을 줄이고 분포를 정규화 -> 변환된 결과를 "Amount_Scaled"이라는 새로운 컬럼으로 맨 앞(0번째 열)에 삽입
    amount_n = np.log1p(df_copy["Amount"])
    df_copy.insert(0, "Amount_Scaled", amount_n)
    
    #불필요한 컬럼 제거
    df_copy.drop(["Time", "Amount"], axis=1, inplace=True)

    # 이상치 제거
    outlier_index = get_outlier(df=df_copy, column="V14", weight=1.5)
    # outlier_index = get_outlier(df=df_copy, column="V14", weight=1.5)
    # outlier_index = get_outlier(df=df_copy, column="V17", weight=1.5)
    # outlier_index = get_outlier(df=df_copy, column="V7", weight=1.5)
    # outlier_index = get_outlier(df=df_copy, column="V20", weight=1.5)
    df_copy.drop(outlier_index, axis=0, inplace=True)
    
    return df_copy

### train, test 로 반환하는 함수

In [ ]:
# 사전 데이터 가공 후 학습과 테스트 데이터 세트를 반환하는 함수

def get_train_test_dataset(df=None):
    df_copy = get_preprocessed_df(df)
    
    X_features = df_copy.iloc[:, :-1]
    y_target = df_copy.iloc[:, -1]
    
    X_train, X_test, y_train, y_test = train_test_split(X_features, y_target,
                                                        test_size=0.3, random_state=0, stratify=y_target)
    
    return X_train, X_test, y_train, y_test

In [ ]:
# 사전 데이터 가공 후 학습과 테스트 데이터 세트를 반환하는 함수

def get_train_test_dataset2(df=None):
    df_copy = get_preprocessed_df(df)
    
    X_features = df_copy.iloc[:, :-1]
    y_target = df_copy.iloc[:, -1]
    
    X_train2, X_test2, y_train2, y_test2 = train_test_split(X_features, y_target,
                                                        test_size=0.3, random_state=0, stratify=y_target)
    
    return X_train2, X_test2, y_train2, y_test2

## 모델 평가 함수

In [ ]:
# 평가 함수

def get_clf_eval(y_test, pred, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    
    print("오차 행렬")
    print(confusion)
    print(f"정확도: {accuracy:.4f}, 정밀도: {precision: .4f}, 재현율: {recall: .4f}, f1스코어: {f1:.4f}, roc-auc: {roc_auc:.4f}")

### 모델 학습 및 예측 -> 평가 함수 호출

In [ ]:
def get_model_train_eval(model, ftr_train=None, ftr_test=None, tgt_train=None, tgt_test=None):
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    pred_proba = model.predict_proba(ftr_test)[:, 1]
    get_clf_eval(tgt_test, pred, pred_proba)

In [ ]:
def get_model_train_eval2(model, ftr_train=None, ftr_test=None, tgt_train=None, tgt_test=None):
    model.fit(ftr_train, tgt_train,
              eval_set=[(X_test, y_test)],
              verbose=False
             )
    pred = model.predict(ftr_test)
    pred_proba = model.predict_proba(ftr_test)[:, 1]
    get_clf_eval(tgt_test, pred, pred_proba)

### 이상치 제거 함수 

In [ ]:
def get_outlier(df=None, column=None, weight=1.5):
    fraud = df[df["Class"]==1][column]
    quantile_25 = np.percentile(fraud.values, 25)
    quantile_75 = np.percentile(fraud.values, 75)
    
    iqr = quantile_75 - quantile_25
    iqr_weight = iqr * weight
    lowest_val = quantile_25 - iqr_weight
    highest_val = quantile_75 + iqr_weight
    
    outlier_index = fraud[(fraud < lowest_val) | (fraud > highest_val)].index
    return outlier_index

### 변수간 상관관계

In [ ]:
import seaborn as sns
plt.figure(figsize=(9,9))
corr = train.corr()
sns.heatmap(corr, cmap='RdBu')
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = get_train_test_dataset(train)

In [ ]:
X_train.head()

In [ ]:
X_train2, X_test2, y_train2, y_test2 = get_train_test_dataset2(train)

In [ ]:
amount_scaled_col = X_train2.pop("Amount_Scaled")
X_train2["Amount_Scaled"] = amount_scaled_col

amount_scaled_col = X_test2.pop("Amount_Scaled")
X_test2["Amount_Scaled"] = amount_scaled_col

In [ ]:
X_train2.head(3)

In [ ]:
print("학습 데이터 레이블 값 비율")
print(y_train.value_counts()/y_train.shape[0] * 100)
print()
print("테스트 데이터 레이블 값 비율")
print(y_test.value_counts()/y_test.shape[0] * 100)

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)

# X_train_over, y_train_over = smote.fit_resample(X_train, y_train)
X_train2_over, y_train2_over = smote.fit_resample(X_train2, y_train2)

In [ ]:
# print("SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ", X_train.shape, y_train.shape)
# print("\nSMOTE 적용 후 학습용 피처/레이블 데이터 세트: ", X_train_over.shape, y_train_over.shape)
print("SMOTE2 적용 전 학습용 피처/레이블 데이터 세트: ", X_train2.shape, y_train2.shape)
print("\nSMOTE2 적용 후 학습용 피처/레이블 데이터 세트: ", X_train2_over.shape, y_train2_over.shape)

# print("\nSMOTE 적용 후 레이블 값 분포: \n", pd.Series(y_train_over).value_counts(normalize=True))
print("\nSMOTE2 적용 후 레이블 값 분포: \n", pd.Series(y_train2_over).value_counts(normalize=True))

## Ensemble

In [ ]:
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

model1 = XGBClassifier()
model2 = LGBMClassifier()
model3 = LogisticRegression()

voting_clf = VotingClassifier(
    estimators=[('xgb', model1), ('lgbm', model2), ('lr', model3)],
    voting='soft'  # 확률 기반 앙상블
)
voting_clf.fit(X_train, y_train)

# 검증용 데이터로 확률값 예측 (클래스 1의 확률)
voting_pred_proba = voting_clf.predict_proba(X_test)[:, 1]

# ROC-AUC 평가
roc_auc = roc_auc_score(y_test, voting_pred_proba)
print("VotingClassifier ROC-AUC: {:.4f}".format(roc_auc))


## XGBoost

In [ ]:
from xgboost import XGBClassifier
xgboost = XGBClassifier(random_state=0)
xgboost2_clf = XGBClassifier(random_state=0)

### xgboost + smote 적용 전

In [ ]:
get_model_train_eval(model = xgboost, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

### xgboost + smote 적용 후

In [ ]:
get_model_train_eval(model = xgboost2_clf, ftr_train=X_train2_over, ftr_test=X_test2, tgt_train=y_train2_over, tgt_test=y_test2)

In [ ]:
X_train2_over.head(3)

### xgboost + smote 적용 후 + 하이퍼 파라미터 튜닝
    n_estimators=900,
    max_depth=5,
    learning_rate=0.02,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,  # 불균형 조정
정확도: 0.9973, 정밀도:  0.4299, 재현율:  0.8598, f1스코어: 0.5732, roc-auc: 0.9779

In [ ]:
from xgboost import XGBClassifier
from collections import Counter
counter = Counter(y_train2)
scale_pos_weight = counter[0] / counter[1]
print(f'counter[0]: {counter[0]:.4f}, counter[1]: {counter[1]:.4f}')
print(f'scale_pos_weight: {scale_pos_weight:.4f}')

xgboost2 = XGBClassifier(
    n_estimators=900,
    max_depth=5,
    learning_rate=0.02,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,  # 불균형 조정
    use_label_encoder=False,
    eval_metric='auc',
    early_stopping_round=100
    
)
get_model_train_eval(model = xgboost2, ftr_train=X_train2_over, ftr_test=X_test2, tgt_train=y_train2_over, tgt_test=y_test2)

## 로지스틱 회귀

In [ ]:
lr_clf = LogisticRegression(max_iter=1000)
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average=False)

In [ ]:
get_model_train_eval(model = lr_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

In [ ]:
get_model_train_eval(model = lr_clf, ftr_train=X_train2_over, ftr_test=X_test2, tgt_train=y_train2_over, tgt_test=y_test2)

In [ ]:
# V17 이상치 제거 후 선형 회귀
get_model_train_eval(model = lr_clf, ftr_train=X_train2, ftr_test=X_test2, tgt_train=y_train2, tgt_test=y_test2)

## LightGBM

In [ ]:
get_model_train_eval(model = lgbm_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

In [ ]:
get_model_train_eval(model = lgbm_clf, ftr_train=X_train2_over, ftr_test=X_test2, tgt_train=y_train2_over, tgt_test=y_test2)

In [ ]:
# 똑같은 전처리 적용
def preprocess_test_df(df):
    df_processed = df.copy()
    df_processed.insert(0, 'Amount_Scaled', np.log1p(df_processed['Amount']))
    df_processed.drop(['Time', 'Amount'], axis=1, inplace=True)
    return df_processed

test_processed = preprocess_test_df(test)
test_processed.head(3)

In [ ]:
# 'Amount_Scaled' 컬럼을 가장 마지막으로 이동
cols = [col for col in test_processed.columns if col != 'Amount_Scaled'] + ['Amount_Scaled']
test_processed = test_processed[cols]

# 결과 확인
test_processed.head(3)

In [ ]:
submission = pd.read_csv("/kaggle/input/modu-ds-4-credit-card-fraud-detection/sample_submission.csv")

submission['Class'] = xgboost2_clf.predict_proba(test_processed)[:, 1]

submission.to_csv("submission.csv", index=False)

In [ ]:
submission.to_csv("./submission.csv", index=False)

In [ ]:
# submission = pd.read_csv("/kaggle/input/modu-ds-4-credit-card-fraud-detection/sample_submission.csv")

In [ ]:
# test["Amount_Scaled"] = np.log1p(test["Amount"])

In [ ]:
# test.drop(["Time", "Amount"], axis=1, inplace=True)

In [ ]:
# test.head(3)

In [ ]:
# train.head(3)

In [ ]:
# lgbm_pred = lgbm_clf.predict(test)
# xgboost2_pred = xgboost2_clf.predict(test)

In [ ]:
# lgbm_pred_proba = lgbm_clf.predict_proba(test)
# xgboost2_pred_proba = xgboost2_clf.predict_proba(test)

In [ ]:
# test["Class"] = lgbm_pred
# test["Class"] = xgboost2_pred

In [ ]:
# test.Class.value_counts()

In [ ]:
# del submission["Class"]

In [ ]:
# test["Class"] = lgbm_pred_proba
# test["Class"] = xgboost2_pred_proba

In [ ]:
# submission = submission.merge(test[["id", "Class"]], on="id")

In [ ]:
# submission.to_csv("./submission.csv", index=False)